In [1]:
import redis

In [34]:
r = redis.Redis(host="localhost", port=6379, db=0)
r.mset({"foo": "bar", "baz": "qux"})
r.get("foo")

b'bar'

In [33]:
import datetime
stoday = datetime.date.today().isoformat()
r.sadd(stoday, "foo", "baz")

2

In [9]:
r.smembers(stoday)

{b'baz', b'foo'}

In [35]:
import random
random.seed(444)

hats = {f"hat: {random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0
    },
    {

        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0
    }
)}

In [36]:
hats

{'hat: 1326692461': {'color': 'black',
  'price': 49.99,
  'style': 'fitted',
  'quantity': 1000,
  'npurchased': 0},
 'hat: 1236154736': {'color': 'maroon',
  'price': 59.99,
  'style': 'hipster',
  'quantity': 500,
  'npurchased': 0},
 'hat: 56854717': {'color': 'green',
  'price': 99.99,
  'style': 'baseball',
  'quantity': 200,
  'npurchased': 0}}

In [19]:
r = redis.Redis(db=1)
import json

In [39]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        print(h_id)
        pipe.hset(h_id, mapping=hat)
        print(pipe)
    pipe.execute()

hat: 1326692461
Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>
hat: 1236154736
Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>
hat: 56854717
Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>


In [44]:
r.hincrby("hat: 123456789", "npurchased", 1)

3

In [40]:
r.keys()

[b'hat: 56854717', b'hat: 1236154736', b'hat: 1326692461']

In [41]:
r.set("a", "b")
r.keys()

[b'hat: 56854717', b'hat: 1236154736', b'a', b'hat: 1326692461']

In [47]:
import logging

logging.basicConfig()

class OutOfStockError(Exception):
    """Raised when a hat is out of stock"""

from time import sleep 

def buyitem(rd, itemid):
    with r.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                pipe.watch(itemid) # raise watch if exception raised 
                nleft = r.hget(itemid, "quantity")
                if int(nleft) > 0:
                    pipe.multi() # Take the look 
                    pipe.hincrby(itemid, "quantity", -1)
                    pipe.hincrby(itemid, "npurchased", 1)
                    sleep(50)
                    pipe.execute()
                    break
                else:
                    pipe.unwatch()
                    raise OutOfStockError()
            except redis.WatchError:
                error_count += 1
                logging.warning("WatchError #%d: %s", error_count, itemid)
                    

In [48]:
buyitem(r, 'hat: 56854717')

In [49]:
r.hmget("hat: 56854717", "quantity", "npurchased")

[b'2999', b'1']

In [50]:
# Key value expiration
from datetime import timedelta

r.setex(
    "runner",
    timedelta(seconds=30),
    value="I'm running out of time"
)

True

In [89]:
r.setex("a", timedelta(seconds=3000), value="b")


True

In [90]:
r.expire("a", timedelta(seconds=3))
r.persist("a")

True

In [91]:
r.ttl("a")

-1